<a href="https://colab.research.google.com/github/sirius70/NLP_HW4/blob/main/decoding_(3)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Model Decoding Algorithms

In this notebook, you will implement several language model decoding algorithms, from simple greedy decoding to beam search. You will test these algorithms using GPT-2 as implemented in the huggingface `transformers` library. You will evaluate the results of an open-ended story generation task, where the goal is not to match a particular human-written reference text and more to generate text that is fluent and diverse. This gives more scope to different decoding approaches.

The `transformers` library implements most of these decoding algorithms, but you will be implementing them yourself using the capabilities of the underlying pytorch library.

This code will benefit from running on a GPU.  If you're running on Colab, **choose "Runtime Type" = GPU for running this notebook (Runtime > Change Runtime Type > T4 GPU).**

The evaluation and harness code for beam search is based on an earlier assignment by Jaehun Jung, Gary Jiacheng, and Yejin Choi from the University of Washington.

## Setup

Please make sure to run all cells in the setup section, even though you won't implement anything yet. First, we install the required libraries.

In [ ]:
!pip install transformers==4.38.2
!pip install datasets
!pip install evaluate

Now we check available devices and set the randomizer seed for replicability.

In [ ]:
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

def set_seed(seed=19260817):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

device: cuda


We load a dataset of prompts for open-ended story generation derived from the [ROCStories corpora](https://cs.rochester.edu/nlp/rocstories/).

In [ ]:
from datasets import load_dataset

dataset = load_dataset('Ximing/ROCStories')
train_data, dev_data, test_data = dataset['train'], dataset['validation'], dataset['test']

print(train_data[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'story_id': '080198fc-d0e7-42b3-8e63-b2144e59d816', 'prompt': 'On my way to work I stopped to get some coffee.', 'continuation': 'I went through the drive through and placed my order. I paid the cashier and patiently waited for my drink. When she handed me the drink, the lid came off and spilled on me. The coffee hurt and I had to go home and change clothes.', 'constraint_words': ['drive', 'order', 'drink', 'lid', 'coffee', 'hurt', 'home', 'change', 'clothes']}


Unlike more constrained generation tasks such as translation and summarization, we cannot usefully check the generated output against a single human reference. Instead, we evaluate its overall fluency, diversity, and similarity to other English texts.

To score the English **fluency** of generated sentences, we will use the CoLA classifier.  This is a RoBERTa-large classifier trained on the CoLA corpus (Warstadt et al., 2019), which contains sentences paired with grammatical acceptability judgments.

In [ ]:
from evaluate import load
from transformers import RobertaForSequenceClassification, RobertaTokenizer

perplexity_scorer = load("perplexity", module_type="metric")
cola_model_name = "textattack/roberta-base-CoLA"
cola_tokenizer = RobertaTokenizer.from_pretrained(cola_model_name)
cola_model = RobertaForSequenceClassification.from_pretrained(cola_model_name).to(device)

def batchify(data, batch_size):
    assert batch_size > 0

    batch = []
    for item in data:
        # Yield next batch
        if len(batch) == batch_size:
            yield batch
            batch = []

        batch.append(item)

    # Yield last un-filled batch
    if len(batch) != 0:
        yield batch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


To evaluate **diversity**, we will simply count the number of distinct 1-, 2-, and 3-grams in the output.

To evaluate **naturalness**, or similarity to other English texts, we will use the perplexity of the output under the model, i.e., GPT-2. Think about which decoding method you would expect to perform the best on this metric.

In [ ]:
def compute_perplexity(texts, model='gpt2', batch_size=8):
    score = perplexity_scorer.compute(predictions=texts, add_start_token=True, batch_size=batch_size, model_id=model)
    return score['mean_perplexity']


def compute_fluency(texts, batch_size=8):
  scores = []
  for b_texts in batchify(texts, batch_size):
    inputs = cola_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
      logits = cola_model(**inputs).logits
      probs = logits.softmax(dim=-1)
      scores.extend(probs[:, 1].tolist())
  return sum(scores) / len(scores)


def compute_diversity(texts):
    unigrams, bigrams, trigrams = [], [], []
    total_words = 0
    for gen in texts:
        o = gen.split(' ')
        total_words += len(o)
        for i in range(len(o)):
            unigrams.append(o[i])
        for i in range(len(o) - 1):
            bigrams.append(o[i] + '_' + o[i + 1])
        for i in range(len(o) - 2):
            trigrams.append(o[i] + '_' + o[i + 1] + '_' + o[i + 2])
    return len(set(unigrams)) / len(unigrams), len(set(bigrams)) / len(bigrams), len(set(trigrams)) / len(trigrams)


def evaluate(generations, experiment):
  generations = [_ for _ in generations if _ != '']
  perplexity = compute_perplexity(generations)
  fluency = compute_fluency(generations)
  diversity = compute_diversity(generations)
  print(experiment)
  print(f'perplexity = {perplexity:.2f}')
  print(f'fluency = {fluency:.2f}')
  print(f'diversity = {diversity[0]:.2f}, {diversity[1]:.2f}, {diversity[2]:.2f}')
  print()

debug_sents = ["This restaurant is awesome", "My dog is cute and I love it.", "Today is sunny."]
evaluate(debug_sents, 'debugging run')

  0%|          | 0/1 [00:00<?, ?it/s]

debugging run
perplexity = 178.64
fluency = 0.98
diversity = 0.87, 1.00, 1.00



We install GPT-2 and its associated tokenizer from huggingface.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token="<|endoftext|>")
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Batching and Working with Tensors

To improve the throughput of language model training and inference, especially on GPUs, we usually group together inputs in **batches**.  What this means for your generation code is that at each function call, you will decide on the next token for several inputs in parallel. Instead of a single vector of the pre-softmax logits of the next token, you will get a two-dimensional tensor of shape $B \times V$ for batch size $B$ and token vocabulary size $V$.

Consider a toy example of a random $4 \times 5$ matrix $A$.

In [ ]:
A = torch.randn(4, 5)

**TODO**: Familiarize yourself with some of the functions on tensors that will be useful for implementing the different decoding algorithms below: `torch.argmax`, `torch.multinomial`, `torch.nn.functional.softmax`, `torch.squeeze`, `torch.topk` and others. You can consult [pytorch's documentation](https://docs.pytorch.org/docs/stable/index.html) or other resources. These functions and others in the pytorch library are all fine to use in your code below. This cell is not graded.

In [ ]:
# TODO: Familiarize yourself with some pytorch functions by manipulating the matrix A.
print("torch.argmax - Find indices of maximum values")
print("Max index along dim=1 (row-wise):", torch.argmax(A, dim=1))
print("Max index along dim=0 (column-wise):", torch.argmax(A, dim=0))
print("Max index in entire tensor:", torch.argmax(A))


# torch.softmax - convert to probability distribution
print("F.softmax - Convert to probabilities")
probs = F.softmax(A, dim=-1)  # dim=-1 means last dimension (row-wise)
print("Probabilities (each row sums to 1):")
print(probs)
print("Row sums (should all be 1.0):", probs.sum(dim=1))


# torch.multinomial - sample from probability distribution
print("torch.multinomial - Sample from distribution")
samples = torch.multinomial(probs, num_samples=1)
print("Sampled indices (one per row):", samples.squeeze())
samples_multiple = torch.multinomial(probs, num_samples=3, replacement=True)
print("Multiple samples (3 per row):")
print(samples_multiple)


print("\nCumulative probabilities for top-p:")
sorted_probs, sorted_indices = torch.sort(probs[0], descending=True)
cum_probs = torch.cumsum(sorted_probs, dim=-1)
print("Sorted probs:", sorted_probs)
print("Cumulative probs:", cum_probs)
print("Tokens needed for 0.9 probability mass:", (cum_probs <= 0.9).sum() + 1)



torch.argmax - Find indices of maximum values
Max index along dim=1 (row-wise): tensor([0, 1, 2, 3])
Max index along dim=0 (column-wise): tensor([3, 1, 2, 3, 3])
Max index in entire tensor: tensor(6)
F.softmax - Convert to probabilities
Probabilities (each row sums to 1):
tensor([[0.4052, 0.1135, 0.1859, 0.1457, 0.1497],
        [0.1286, 0.4678, 0.2296, 0.1559, 0.0180],
        [0.1142, 0.1958, 0.4274, 0.1892, 0.0733],
        [0.2637, 0.1358, 0.0905, 0.3233, 0.1868]])
Row sums (should all be 1.0): tensor([1.0000, 1.0000, 1.0000, 1.0000])
torch.multinomial - Sample from distribution
Sampled indices (one per row): tensor([0, 1, 2, 1])
Multiple samples (3 per row):
tensor([[0, 1, 0],
        [2, 1, 1],
        [0, 0, 2],
        [0, 4, 1]])

Cumulative probabilities for top-p:
Sorted probs: tensor([0.4052, 0.1859, 0.1497, 0.1457, 0.1135])
Cumulative probs: tensor([0.4052, 0.5911, 0.7409, 0.8865, 1.0000])
Tokens needed for 0.9 probability mass: tensor(5)


## Basic Decoding Algorithms

In this section, you will implement a few basic decoding algorithms:
1. Greedy decoding
2. Simple sampling
3. Temperature sampling
4. Top-k sampling
5. Top-p sampling

We have provided a wrapper function `decode()` that takes care of batching, controlling max length, and handling the EOS token.
You will be asked to implement the core function of each method: *given the pre-softmax logits of the next token, decide what the next token is.*

**The wrapper calls the core function of each decoding algorithm, which you will implement in the subsections below.**

In [ ]:
def decode(prompts, max_len, method, **kwargs):
  encodings_dict = tokenizer(prompts, return_tensors="pt", padding=True)
  input_ids = encodings_dict['input_ids'].to(device)
  attention_mask = encodings_dict['attention_mask'].to(device)

  model_kwargs = {
      'attention_mask': attention_mask,
      'cache_position': torch.arange(input_ids.shape[1], device=device)
  }
  batch_size, input_seq_len = input_ids.shape

  unfinished_sequences = torch.ones(batch_size, dtype=torch.long, device=device)

  for step in range(max_len):
    model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
    with torch.no_grad():
      outputs = model(**model_inputs, return_dict=True, output_attentions=False, output_hidden_states=False)

    if step == 0:
      last_non_masked_idx = torch.sum(attention_mask, dim=1) - 1
      next_token_logits = outputs.logits[range(batch_size), last_non_masked_idx, :]
    else:
      next_token_logits = outputs.logits[:, -1, :]

    log_prob = F.log_softmax(next_token_logits, dim=-1)

    if method == 'greedy':
      next_tokens = greedy(next_token_logits)
    elif method == 'sample':
      next_tokens = sample(next_token_logits)
    elif method == 'temperature':
      next_tokens = temperature(next_token_logits, t=kwargs.get('t', 0.8))
    elif method == 'topk':
      next_tokens = topk(next_token_logits, k=kwargs.get('k', 20))
    elif method == 'topp':
      next_tokens = topp(next_token_logits, p=kwargs.get('p', 0.7))

    # finished sentences should have their next token be a padding token
    next_tokens = next_tokens * unfinished_sequences + tokenizer.pad_token_id * (1 - unfinished_sequences)

    input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
    model_kwargs = model._update_model_kwargs_for_generation(outputs, model_kwargs, is_encoder_decoder=model.config.is_encoder_decoder)

    # if eos_token was found in one sentence, set sentence to finished
    unfinished_sequences = unfinished_sequences.mul((next_tokens != tokenizer.eos_token_id).long())

    if unfinished_sequences.max() == 0:
      break

  response_ids = input_ids[:, input_seq_len:]
  response_text = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in response_ids]

  return response_text

In [ ]:
# For debugging, we duplicate a single prompt 10 times so that we obtain 10 generations for the same prompt
dev_prompts = [dev_data[0]['prompt']] * 10

def print_generations(prompts, generations):
  for prompt, generation in zip(prompts, generations):
    print(f'{[prompt]} ==> {[generation]}')

### Greedy Decoding

The simplest strategy is to select the token with the highest probability.

In [ ]:
# TODO: Implement greedy decoding

def greedy(next_token_logits):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # Compute `next_tokens` from `next_token_logits`.
  next_tokens = torch.argmax(next_token_logits,dim = -1)

  return next_tokens


generations = decode(dev_prompts, max_len=20, method='greedy')
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m goin

**TODO**: What do you observe when the code above generates 10 times from the test prompt?

Greedy decoding produces identical,repetitive outputs across all generations.Since it selects the highest probabaility token at each step,there is no variation between samples- all 10 generations are eaxctly the same,the model gets stuck in a loop repeating the same phrase pattern.This happens because greedy decoding always pick the most probabale next token,so once it gets into a pattern,it keeps repeating it deterministically with no variation.

### Simple Sampling and Temperature Sampling

Greedy decoding has some drawbacks that we discussed in class. One alternative is simply to sample from the probability distribution of the language model. We'll call that _simple sampling_ to distinguish it from more complicated methods.

In [ ]:
# TODO: Implement simple sampling.
def sample(next_token_logits):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # Compute the probabilities from the logits,
  # then compute `next_tokens` from `probs`.
  # Hint: `probs` should have size (B, V)
  probs = F.softmax(next_token_logits,dim = -1)

  next_tokens = torch.multinomial(probs,num_samples =1).squeeze(-1)

  return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='sample')
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\nKiddiak said he stepped on it, hit the brakes hard and decided to cross first']
['Ryan was called by his friend to skip work one day.'] ==> [" Their interaction didn't work at all by the time he would get back to Louisville. Some details that"]
['Ryan was called by his friend to skip work one day.'] ==> [' According to investigators at the time, Gianfranco was yelling at them for not doing enough work']
['Ryan was called by his friend to skip work one day.'] ==> [' "No, my boyfriend doesn\'t do work," Cat started, desperate to dial a phone in distress']
['Ryan was called by his friend to skip work one day.'] ==> [' Later, he did not show up for school again until late the next day. After catching a train']
['Ryan was called by his friend to skip work one day.'] ==> [' The fliers denounced Gov. Pat Brown for having insufficient police presence in the building, and he engineered']
['Ryan was called by his friend to skip wo

**TODO**: What do you observe when you generating 10 times from the test prompt?

Simple Scaling produces diverse,varied outputs across all generations,unlike greedy decoding,samples tokens from the probability distribution,introducing randomness that prevents repetitive patterns and generates unique text each time.While diversity  increase,some outputs,may be less,coherent or drift off topic.Each of the 10 generations produces completely different text -no outputs are the same.



The probability distribution of the model has some undesirable properties, as we discussed in class. You can adjust its entropy using a _temperature_ parameter `t`.

In [ ]:
# TODO: Implement temperature-scaled sampling.

def temperature(next_token_logits, t):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  - t: float
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''
  scaled_logits = next_token_logits/t
  probs = F.softmax(scaled_logits,dim = -1)
  next_tokens = torch.multinomial(probs,num_samples=1).squeeze(-1)

  return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='temperature', t=0.8)
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\nKiddiak said he stepped on the blood spur out of pity from his cousin Jimmie']
['Ryan was called by his friend to skip work one day.'] ==> [" Their interaction didn't go well.\n\nHerad's mother was in a panic. Unable to"]
['Ryan was called by his friend to skip work one day.'] ==> [" He said he hadn't asked if he'd be coming back to work for a week.\n\n"]
['Ryan was called by his friend to skip work one day.'] ==> [' "I told my boss I was leaving work early to take care of my sons\' homework. So']
['Ryan was called by his friend to skip work one day.'] ==> [' He also said he was going to play out of town because the game was cancelled. Johnson also said']
['Ryan was called by his friend to skip work one day.'] ==> [' The next morning, after talking to other parents who also work in the area, he was told to']
['Ryan was called by his friend to skip work one day.'] ==> [' It was before he was going to college.\n\n"Oh, I\'l

**TODO**: What value of `t` would be equivalent to greedy decoding? What value of `t` would be equivalent to simple sampling?

Greedy decoding t -> 0 as it approaches zero.As temperature approaches 0,the softmax becomes more peaked,making the highest probability token approach probability 1.This effectively makes the samppling deterministic,where it always picks argmax.

Simple sampling t = 1.0
Temperature of 1 leaves the logits unchanged,and then giving us the original probability distribution from the model for sampling.

Makes distribution flatter more uniform where it is more random or creative. when t is greater than 1.

### Top-k Sampling

By Zipf's law, the distribution over tokens contains many low-probability words in the tail. It might be useful to sample only from the most probable words.

In [ ]:
# TODO: Implement top-k sampling

def topk(next_token_logits, k):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  - k: int
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # Keep only top-k tokens with highest probabilities.
  # Hint: Create a mask to zero out all logits not in top-k
  top_k_logits,top_k_indices = torch.topk(next_token_logits,k,dim =-1)
  mask = torch.full_like(next_token_logits,float('-inf'))
  mask.scatter_(-1,top_k_indices,top_k_logits)

  probs = F.softmax(mask,dim = -1)
  next_tokens = torch.multinomial(probs,num_samples=1).squeeze(-1)

  return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='topk', k=20)
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I wasn\'t sure what to expect, so he asked me why I didn\'t work']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I\'m tired of talking about this because it\'s a big distraction. I\'m tired']
['Ryan was called by his friend to skip work one day.'] ==> [' When he arrived at work, the other workers were on the line and asked if he would be willing']
['Ryan was called by his friend to skip work one day.'] ==> [' "Why don\'t you try to do work? You don\'t look like a person," his friend']
['Ryan was called by his friend to skip work one day.'] ==> [' He said he did not have enough money to pay his bills. He told his girlfriend, a teacher']
['Ryan was called by his friend to skip work one day.'] ==> [' The next morning, after work, he would get back to his job, and the next day,']
['Ryan was called by his friend to skip work one day.'] ==> [" It was the first time they had ever met, and they couldn't qui

**TODO**: What value of `k` makes this equivalent to simple sampling?

When k equals the full vocabulary size,all tokens are kept in the top-k ,so no tokens were masked out .This makes top-k sampling equivalent to simple sampling since we are sampling from the complete probabaility distribution.When k equals the vocabulary size(V) ,the torch.topk()nfunction returns all tokens ,so the mask does not filter anything out,We ended from the complete original probabaility distribution which is exactly what simple sampling does.Small k as only top 20 most probable tokens are considered ,making the generation more focused and conservative.


### Top-p Sampling

But should we sample from the same number of candidates in all contexts? Maybe in different contexts the number of good candidates is larger or smaller.

In [ ]:
# TODO: Implement top-p (nucleus) sampling.

def topp(next_token_logits, p):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  - p: float
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # TODO: Sort the logits in descending order, and compute
  # the cumulative probabilities `cum_probs` on the sorted logits
  # and then sample from those that make up the top p probability mass.
  sorted_logits,sorted_indices = torch.sort(next_token_logits,descending=True)
  sorted_probs = F.softmax(sorted_logits,dim = -1)
  cum_probs = torch.cumsum(sorted_probs,dim = -1)

  sorted_indices_to_remove = cum_probs > p
  sorted_indices_to_remove[...,1:] = sorted_indices_to_remove[...,:-1].clone()
  sorted_indices_to_remove[...,0] = False

  sorted_logits[sorted_indices_to_remove] = float('-inf')

  mask = torch.full_like(next_token_logits,float('-inf'))
  mask.scatter_(-1,sorted_indices,sorted_logits)

  probs = F.softmax(mask,dim = -1)

  next_tokens = torch.multinomial(probs,num_samples = 1).squeeze(-1)

  return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='topp', p=0.7)
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ["\n\nKiddie said he also didn't know if he was working out or not, and"]
['Ryan was called by his friend to skip work one day.'] ==> [' On the day of the attack, he took the opportunity to share a beautiful moment with her, a']
['Ryan was called by his friend to skip work one day.'] ==> [' When he arrived at work, the new tenant was coming in with a pile of trash.\n\n']
['Ryan was called by his friend to skip work one day.'] ==> [' "No, my boyfriend doesn\'t do that," he said, before jogging out to his room']
['Ryan was called by his friend to skip work one day.'] ==> [' He also sat in a wheelchair at his hospital bed. In the room was a young woman who looked']
['Ryan was called by his friend to skip work one day.'] ==> [" The rookie's parents said he was talking to his father and asking him to help him.\n\n"]
['Ryan was called by his friend to skip work one day.'] ==> [" Then he goes back to work. He can't afford to do it. S

**TODO**: What value of `p` makes this equivalent to simple sampling?

When p is 1.0,the cumulative proabability threshold includes all tokens,100 percent of the probability mass,so no tokens are filtered out.This makes top k sampling equivalent to simple sampling since we are sampling from the complete probability distribution.When p is 1.0 cumulative probability  it never exceeds 1.0 so cum_probs is greater than p is always False.So this means no tokens are removed and sample from the complete vocabulary distribution.



### Evaluate!

Run the following cell to obtain the evaluation results, which you should include in your writeup.
Also don't forget to answer the questions.

In [ ]:
prompts = [item['prompt'] for item in test_data][:10]
GENERATIONS_PER_PROMPT = 10
MAX_LEN = 100

for experiment in ['greedy', 'sample', 'temperature', 'topk', 'topp']:
  generations = []
  for prompt in tqdm(prompts):
    generations += decode([prompt] * GENERATIONS_PER_PROMPT, max_len=MAX_LEN, method=experiment)
  evaluate(generations, experiment)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

greedy
perplexity = 2.08
fluency = 0.78
diversity = 0.01, 0.02, 0.03



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

topk
perplexity = 12.24
fluency = 0.74
diversity = 0.26, 0.75, 0.96



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

topp
perplexity = 12.33
fluency = 0.72
diversity = 0.29, 0.76, 0.96



**TODO**: Which method has the best and worst perplexity? Fluency? Diversity?

Best perplexity has the greedy (2.08) lowest perplexity,but it most predictable.The worst perplexity  is the sample (70.58)which has the highest perplexity and least predictable.The best fluency is the greedy (0.78) with the highest perplexity,least predictable.The best diversity is the sample (0.43,0.90,0.99) whiich has the highest diversity scores.The worst diversity is the greedy (0.01,0.02,0.03) where it has no diversity and also not  repetitive .

Greedy has the excellent perplexity and fluency but terrible diversity where the sample has exccellent diversity where it has poor perplexity and fluency .Temperature,top k,top p offer balanced middle grounds.

## Beam Search

In this part of the assignment, your main task is to implement beam search algorithm. While debugging for implementation, we recommend setting `device` to `cpu`, as debugging would not require too much resource for this assignment. Once you are done with implementation, you may switch `device` to `cuda` and choose `Colab Runtime Type = GPU` for faster evaluation of your algorithm.

We have provided some helpful functions and classes. Please make sure that you understand them. But you don't need to implement/change anything of them until the code block marked TODO.

### Configurations: load model and tokenizer

In [ ]:
from dataclasses import dataclass
from typing import List, Tuple

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token="<|endoftext|>")
tokenizer.padding_side = "left"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.eval()

pad_token_id = tokenizer.pad_token_id
eos_token_id = tokenizer.eos_token_id

### Helper classes: `BeamHypothesisList` and `BeamManager`

In [ ]:
@dataclass
class BeamHypothesis:
    def __init__(self, input_ids: torch.LongTensor, score: torch.FloatTensor | float):
        self.input_ids: torch.LongTensor = input_ids  # a single token sequence of size (seq_len,)
        self.score: torch.FloatTensor = score  # a scalar score for the token sequence

    def __str__(self):
        return f"BeamHypothesis(input_ids: {self.input_ids}, score: {self.score})"

class BeamHypothesisList:
    def __init__(self, beam_width: int):
        self.beam_hypotheses: List[BeamHypothesis] = []  # list of beam_hypothesis
        self.beam_width: int = beam_width

        self.worst_score = 1e9  # worst beam score in self.beam_hypotheses

    def add(self, new_input_ids: torch.LongTensor, sum_logprobs: float):
        """
        :param new_input_ids: new token sequence of size (1, seq_len)
        :param sum_logprobs: sum of log probabilities of tokens in new_input_ids
        Given a new hypothesis (new_input_ids) and its corresponding sum_logprobs, update self.beam_hypotheses with a finished hypothesis.
        (1) If the new_input_ids has higher score than the current worst score in self.beam_hypotheses,
            we replace the worst one with the new hypothesis.
        (2) Otherwise, we do not make any change to self.beam_hypotheses.
        """
        # For score, we compute average token log probability
        score = sum_logprobs / new_input_ids.size(-1)

        # add the new hypothesis if we still have vacant beams
        if len(self.beam_hypotheses) < self.beam_width:
            # Initialize the new_beam_hypothesis using new_input_ids and score
            new_beam_hypothesis: BeamHypothesis = BeamHypothesis(input_ids=new_input_ids, score=score)

            # Add new_beam_hypothesis to beam_hypotheses
            self.beam_hypotheses.append(new_beam_hypothesis)

        # even if the beam_hypotheses are full, if the new hypothesis has higher score than the worst hypothesis, we replace the worst hypothesis
        elif score > self.worst_score:
            # Remove the worst hypothesis, the one with the lowest score in self.beam_hypotheses
            worst_hypothesis: BeamHypothesis = min(self.beam_hypotheses, key=lambda hyp: hyp.score)
            self.beam_hypotheses.remove(worst_hypothesis)

            # Add new hypothesis
            # Initialize the new_beam_hypothesis using new_input_ids and score
            new_beam_hypothesis = BeamHypothesis(input_ids=new_input_ids, score=score)

            # Add new_beam_hypothesis to beam_hypotheses
            self.beam_hypotheses.append(new_beam_hypothesis)

        # Update the worst score - the lowest score among all beams in self.beam_hypotheses
        self.worst_score = min(float(hyp.score) for hyp in self.beam_hypotheses)

        # Sanity check
        assert len(self.beam_hypotheses) <= self.beam_width

In [ ]:
class BeamManager:
    def __init__(self, batch_size: int, beam_width: int):
        self.finished_beam_hypotheses_list = [BeamHypothesisList(beam_width) for _ in range(batch_size)]
        self.batch_size = batch_size
        self.beam_width = beam_width

    def process(self,
                input_ids: torch.LongTensor,
                top_token_scores: torch.FloatTensor,
                top_token_indices: torch.LongTensor,
                top_token_beam_indices: torch.LongTensor
                ):
        """
        :param input_ids: (batch_size * beam_width, current_seq_length), the input_ids that were used to compute top_tokens
        :param top_token_scores: (batch_size, 2 * beam_width), representing the score of each top token
        :param top_token_indices: (batch_size, 2 * beam_width), representing each token's index (in vocabulary) of the top tokens
        :param top_token_beam_indices: (batch_size, 2 * beam_width), representing each token's corresponding beam index of the top tokens

        Note: the input arguments `top_token_*` for each sample in batch are sorted from the largest score to the smallest score.
        For example, if batch_size = 2 and beam_width = 3, then each of these values denote
        top_token_indices[1, 2]: what is the third-best next token for the second sample in the batch?
        top_token_scores[0, 1]: what is the score of the second-best next token for the first sample in the batch?
        top_token_beam_indices[0, 1]: which beam did we use to generate the second-best next token for the first sample in the batch?

        In this function, for each of the top-(2 * beam_width) tokens, we do the following:
        (1) If the top token is EOS token:
            This means that this hypothesis is done. Therefore, we save the hypothesis so-far to self.finished_beam_hypotheses_list.
        (2) If the top token is not EOS token
            We have to keep searching with this hypothesis. Therefore, we prepare the hypothesis for next time step.

        Returns a dictionary, where
        "unfinished_scores": size (batch_size * beam_width,), the score of the unfinished beams
        "unfinished_token_indices": size (batch_size * beam_width,), the index of the last token in the unfinished beams
        "unfinished_beam_indices": the index of the beam that was used to generate the new unfinished beam
        """
        device = top_token_scores.device

        # Initialize unfinished_token_*, which we will return for the next time step.
        unfinished_scores = torch.zeros((self.batch_size, self.beam_width), dtype=top_token_scores.dtype).to(device)  # score of the unfinished beams
        unfinished_token_indices = torch.zeros((self.batch_size, self.beam_width), dtype=top_token_indices.dtype).to(
            device)  # index of the last token of the unfinished beams
        unfinished_token_beam_indices = torch.zeros((self.batch_size, self.beam_width), dtype=top_token_beam_indices.dtype).to(
            device)  # index of the unfinished beam in the batch

        # Loop over the batch
        for batch_idx in range(self.batch_size):
            # Get sample_beam_hypothesis_list: the finished_beam_hypothesis_list for this sample in the batch
            sample_beam_hypothesis_list: BeamHypothesisList = self.finished_beam_hypotheses_list[batch_idx]

            # Get the top_token_scores, top_token_indices, top_token_beam_indices for this sample in the batch
            # NOTE: size of sample_top_token_*: (2 * beam_width,)
            sample_top_token_scores = top_token_scores[batch_idx]
            sample_top_token_indices = top_token_indices[batch_idx]
            sample_top_token_beam_indices = top_token_beam_indices[batch_idx]

            # Loop over all top tokens
            sample_beam_idx = 0
            for top_token_score, top_token_index, top_token_beam_index in zip(
                    sample_top_token_scores, sample_top_token_indices, sample_top_token_beam_indices
            ):
                # Note that top_token_beam_indices only denotes the index of the beam in each sample.
                # We transform this into `beam_idx_in_batch`, where we denote the index of the beam among all (batch_size * beam_width) beams in the batch.
                beam_idx_in_batch = batch_idx * self.beam_width + top_token_beam_index

                # if top_token == EOS, we add the generation so-far to the beam_hypotheses_list
                if top_token_index.item() == eos_token_id:
                    # Among the (batch_size * beam_width) input_ids, find the input_ids that correspond to this top_token
                    # NOTE: the size of new_input_ids: (seq_len,)
                    new_input_ids = input_ids[beam_idx_in_batch]

                    # Add the new beam to sample_beam_hypothesis_list
                    sample_beam_hypothesis_list.add(
                        new_input_ids,
                        top_token_score
                    )

                # if top_token =/= EOS, we aggregate them for next time step.
                else:
                    # Store the score, token_index, beam_idx_in_batch to the unfinished_scores, unfinished_token_indices, unfinished_token_beam_indices
                    unfinished_scores[batch_idx, sample_beam_idx] = top_token_score
                    unfinished_token_indices[batch_idx, sample_beam_idx] = top_token_index
                    unfinished_token_beam_indices[batch_idx, sample_beam_idx] = beam_idx_in_batch

                    sample_beam_idx += 1

                # once we have `beam_width` number of new beams, we don't have to add anymore.
                if sample_beam_idx == self.beam_width:
                    break

        # Return the dictionary of unfinished_scores, unfinished_token_indices, unfinished_beam_indices
        # Make sure to change the size of each tensor to (batch_size * beam_width,)
        return {
            "unfinished_scores": unfinished_scores.view(-1),  # (batch_size * beam_width,)
            "unfinished_token_indices": unfinished_token_indices.view(-1),  # (batch_size * beam_width,)
            "unfinished_beam_indices": unfinished_token_beam_indices.view(-1),  # (batch_size * beam_width,)
        }

    def finalize(
            self,
            input_ids: torch.LongTensor,
            beam_scores: torch.FloatTensor,
    ) -> Tuple[List[torch.LongTensor], List[torch.FloatTensor]]:
        """
        :param input_ids: (batch_idx * beam_width, max_length), input_ids of unfinished beams
        :param beam_scores: (batch_idx * beam_width,), scores of unfinished beams
        Get the final best beams, among
        (1) unfinished beams, for which we get the input_ids and beam_scores as arguments
        (2) finished beams, which we store in self.batch_beam_hypothesis_list
        Returns a tuple of two lists, where
        - tuple[0] is the list of the input_ids of the best beams (length: batch_idx)
        - tuple[1] is the list of the scores of the best beams (length: batch_idx
        """

        # 1. Add all unfinished beam hypotheses to self.finished_beam_hypotheses_list
        for batch_idx in range(self.batch_size):
            # Get sample_beam_hypothesis_list: the finished_beam_hypothesis_list for this sample in the batch
            sample_beam_hypothesis_list: BeamHypothesisList = self.finished_beam_hypotheses_list[batch_idx]

            for sample_beam_idx in range(self.beam_width):
                # Get beam_idx_in_batch: index of the beam in all `batch_size * beam_width` beams in the batch
                beam_idx_in_batch = batch_idx * self.beam_width + sample_beam_idx

                # Get the input_id for this beam, using `beam_idx_in_batch`
                # NOTE: the size of new_input_ids: (seq_len,)
                new_input_ids = input_ids[beam_idx_in_batch]

                # Get the score of this beam, using `beam_idx_in_batch`
                # NOTE: beam_score should be a scalar
                beam_score = beam_scores[beam_idx_in_batch].item()

                # Add the new hypothesis to sample_beam_hypothesis_list
                sample_beam_hypothesis_list.add(new_input_ids, beam_score)

        # 2. Select the best hypothesis from each beam_hypothesis_list
        best_input_ids = []
        best_scores = []
        for batch_idx in range(self.batch_size):
            # Get sample_beam_hypothesis_list: the finished_beam_hypothesis_list for this sample in the batch
            sample_beam_hypothesis_list: BeamHypothesisList = self.finished_beam_hypotheses_list[batch_idx]

            # Get best_hypothesis among sample_beam_hypothesis_list (the one with the highest score)
            best_hypothesis = max(sample_beam_hypothesis_list.beam_hypotheses, key=lambda hyp: hyp.score)

            # Save the input_ids and score of best_hypothesis
            best_input_ids.append(best_hypothesis.input_ids)
            best_scores.append(best_hypothesis.score)

        return best_input_ids, best_scores

### Beam Search

Conceptually, beam search is a straightforward extension to greedy decoding—where we retain the top-k hypotheses instead of the top-1 hypothesis in greedy decoding. However, implementing it is much more complex than the intuition. While there are many “arbitrary” decisions one needs to make to implement beam search, for the purpose of this assignment, we will do the following:

* Compared to greedy decoding, beam search introduces one additional hyper-parameter, beam_width, denoting the number of hypotheses we retain at each time step. For example, set beam_width = 3.

* At each time step, we have access to 3 hypotheses $h_1, h_2, h_3$ generated so far, along with the corresponding scores $s_1, s_2, s_3$ (typically defined as the average log probability of tokens in each hypothesis). To update the hypothesis, we compute next token distribution $p_i$ for each hypothesis $h_i$, then take top-`beam_width` tokens (that makes the highest score when appended to the corresponding hypothesis) across all $p_i$s.

* At each time step, in some of the hypotheses (say $h_1$), we may encounter the EOS token `<EOS>`. In greedy decoding, we can simply terminate the generation process, but in beam search, there still remain the 2 unfinished hypotheses. In this case, we move the finished hypothesis ($h_1$, which encountered the EOS token) into a separate list. For the remaining unfinished hypotheses $h_2$ and $h_3$, we keep searching for the best next tokens, retrieving the top `beam_width` = 3 tokens to create 3 new hypotheses (and their corresponding scores) for the next time step.

* After reaching the max length step, we have two sets of hypotheses—a set of unfinished hypotheses that did not encounter `<EOS>` until the final step, and a set of finished hypotheses. Finally, we compare all hypotheses across the two sets, and return the one with the highest score. Fill in the TODOs in `beam search()`. Given the complexity of the implementation, we provide more
structured skeleton for the algorithm, along with two helper classes `BeamHypothesisList` and `BeamManager`. Note that the two classes are already fully implemented, your job is to finish `beam search()` using the
two helper classes. You may find the [beam search implementation in Huggingface transformers](https://github.com/huggingface/transformers/blob/v4.37.2/src/transformers/generation/utils.py##L2743) useful. In
addition, we leave comments for the expected size of tensor variables as a sanity check.

**TODO: fill in the TODOs in `beam_search`.**

In [ ]:
# TODO: Complete the implementation of beam search.

def beam_search(prompts: List[str], beam_width: int, max_length: int) -> List[str]:
    """
    :param prompts: list of prompt strings
    :param beam_width: number of hypotheses in the beam
    :param max_length: max generation length
    :return: list of generation, including both the original prompt and generation
    """
    # TODO: encode the prompts using tokenizer, to get input_ids and attention_mask
    input_encoding = tokenizer(prompts, return_tensors="pt", padding='longest')
    input_ids = input_encoding['input_ids'].to(device)
    attention_mask = input_encoding['attention_mask'].to(device)

    if input_ids.size(-1) > max_length:
      raise ValueError("Input ID is larger than max_length.")

    # --- Do not change below --- #
    batch_size = input_ids.size(0)
    vocab_size = len(tokenizer)

    # initialize model_kwargs
    model_kwargs = {
        'attention_mask': attention_mask,
        'cache_position': torch.arange(input_ids.shape[1], device=device)
    }

    # interleave input_ids according to beam_width.
    # For example, input_ids for ["Hi", "good"] with beam_width=3 becomes ["Hi", "Hi", "Hi", "good", "good", "good"]
    input_ids, model_kwargs = model._expand_inputs_for_generation(
        input_ids=input_ids,
        expand_size=beam_width,
        is_encoder_decoder=False,
        **model_kwargs,
    )
    # NOTE: the type of `input_ids` and `model_kwargs` are as following:
    # input_ids: tensor of size (batch_size * beam_width, seq_len)
    # model_kwargs: a dictionary with two elements 'attention_mask', sized (batch_size * beam_width, seq_len), and 'cache_position'
    # --- Do not change above --- #

    # TODO: initialize beam_manager
    beam_manager = BeamManager(batch_size,beam_width)

    # TODO: initialize unfinished_beam_scores, a tensor of size (batch_size, beam_width) with all elements = 0
    unfinished_beam_scores = torch.zeros((batch_size,beam_width),dtype=torch.float,device = device)

    # For each sample in the batch, set all initial beam_score to -1e9, except for the first beam
    unfinished_beam_scores[:, 1:] = -1e9
    unfinished_beam_scores = unfinished_beam_scores.view(-1)  # (batch_size * beam_width,)

    while True:
        # --- Do not change below --- #
        model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
        # --- Do not change above --- #

        # TODO: run model forward pass with model_inputs as the input
        # NOTE: we should set return_dict=True, output_attentions=False and output_hidden_states=False
        model_outputs = model(**model_inputs,return_dict = True,output_attentions=False,output_hidden_states=False)


        # TODO compute log_probs for next tokens, using `model_outputs.logits`
        # NOTE: size of next_token_scores: (batch_size * beam_width, vocab_size)
        next_token_logits = model_outputs.logits[:,-1,:]

        next_token_scores = F.log_softmax(next_token_logits,dim = -1)

        # TODO add previous beam_scores to the next_token_scores
        # NOTE: size of new_scores: # (batch_size * beam_width, vocab_size)
        new_scores = next_token_scores + unfinished_beam_scores.unsqueeze(-1)

        # TODO: retrieve top-(2 * beam_width) next tokens for each sample in the batch
        # NOTE: size of `top_token_scores` and `top_token_indices` needs to be: (batch_size, 2 * beam_width)
        new_scores = new_scores.view(batch_size, beam_width * vocab_size)
        # NOTE: `top_token_scores` and `top_token_indices` should be sorted from the one with larget score to the one with smallest score (for each sample in batch)
        # Hint: use torch.topk with largest=True, sorted=True
        # Hint: new_scores needs to be transformed to shape (batch_size, beam_width * vocab_size) prior to topk operation.
        top_token_scores, top_token_indices = torch.topk(new_scores,2*beam_width,dim = -1,largest=True,sorted = True)

        # Since top_token_indices are over beam_width * vocab_size, divide it by beam_width to get vocabulary index and beam index
        top_token_beam_indices = torch.div(top_token_indices, vocab_size,
                                           rounding_mode="floor")  # from which beam the top-token was retrieved from
        top_token_indices = top_token_indices % vocab_size  # the index of top-token in the vocabulary

        # TODO: Run beam_manager.process and save the results in unfinished_beam_scores, unfinished_token_indices and unfinished_beam_indices
        unfinished_beam_outputs = beam_manager.process(input_ids,top_token_scores,top_token_indices,top_token_beam_indices)
        unfinished_beam_scores = unfinished_beam_outputs["unfinished_scores"]
        unfinished_token_indices = unfinished_beam_outputs["unfinished_token_indices"]
        unfinished_beam_indices = unfinished_beam_outputs["unfinished_beam_indices"]

        # --- Prepare input_ids for next time step --- #
        # TODO: index input_ids with the unfinished beam indices
        # NOTE: input_ids should be (batch_size * beam_width, seq_len)
        input_ids = input_ids[unfinished_beam_indices]

        # TODO: concatenate the unfinished_token_indices to the input_ids
        input_ids = torch.cat([input_ids, unfinished_token_indices.unsqueeze(-1)], dim=-1)

        # --- Do not change below --- #
        # update the model_kwargs according to the concatenated input_ids
        model_kwargs = model._update_model_kwargs_for_generation(
            model_outputs, model_kwargs, is_encoder_decoder=False
        )
        if model_kwargs["past_key_values"] is not None:
            model_kwargs["past_key_values"] = model._temporary_reorder_cache(
                model_kwargs["past_key_values"], unfinished_beam_indices,
            )
        # --- Do not change above --- #

        # if unfinished input_ids reach the max seq length, exit the loop
        if input_ids.size(-1) == max_length:
            break

    # TODO: Run beam_manager.finalize to get the best_input_ids and best_scores, among all finished / unfinished beams
    best_input_ids, best_scores = beam_manager.finalize(input_ids,unfinished_beam_scores)

    # if len(best_input_ids) < max_length, pad them to the max length
    for batch_idx, sample_input_ids in enumerate(best_input_ids):
        if sample_input_ids.size(-1) < max_length:
            pad_tensor = torch.LongTensor([pad_token_id] * (max_length - sample_input_ids.size(-1))).to(device)

            # TODO: pad best_input_ids with pad_tensor
            best_input_ids[batch_idx] = torch.cat([sample_input_ids,pad_tensor])

    # TODO: transform best_input_ids (which is currently a list of tensors) into a tensor of size (batch_idx, max_seq_length)
    # Hint: use torch.stack
    best_input_ids = torch.stack(best_input_ids)

    return tokenizer.batch_decode(best_input_ids, skip_special_tokens=True)

### Sanity check for debugging

In [ ]:
sents = [
    "This restaurant is awesome.",
    "My dog is cute and I love it.",
    "Today is sunny.",
]

beam_search(sents, beam_width=5, max_length=15)

# expected output: ["This restaurant is awesome. I've been here a few", 'My dog is cute and I love it.\n\nRated 5 out of', 'Today is sunny. The sun is shining. The']

["This restaurant is awesome. I've been here a few",
 'My dog is cute and I love it.\n\nRated 5 out of',
 'Today is sunny. The sun is shining. The']

### Evaluate Beam Search!

In [ ]:
# If your implementation is efficient enough, the following code will run in no longer than 3 minutes with device = gpu.

from pprint import pprint

prompts = [item['prompt'] for item in test_data][:100]
MAX_LEN = 100
beam_width = 5
BATCH_SIZE = 5

generations = []
for batch_start_idx in tqdm(range(0, len(prompts), BATCH_SIZE)):
  batched_prompts = prompts[batch_start_idx: batch_start_idx + BATCH_SIZE]
  batched_generations = beam_search(batched_prompts, beam_width, MAX_LEN)

  # remove prompt from generation
  batched_generations = [generation[len(prompt):] for prompt, generation in zip(batched_prompts, batched_generations)]

  generations += batched_generations

evaluate(generations, 'Beam Search')

  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipython-input-4107028903.py:51: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  self.worst_score = min(float(hyp.score) for hyp in self.beam_hypotheses)


  0%|          | 0/13 [00:00<?, ?it/s]

Beam Search
perplexity = 2.05
fluency = 0.82
diversity = 0.07, 0.13, 0.17



In [ ]:
# print first 10 generations

sampled_prompts = prompts[:10]
sampled_generations = generations[:10]

for idx, (prompt, generation) in enumerate(zip(sampled_prompts, sampled_generations)):
  print(f"Prompt {idx}")
  print(prompt)
  print(f"Generation {idx}")
  print(generation)
  print("---------")
  print()

Prompt 0
The soccer game was tied 3 to 3 and there was a minute left to play.
Generation 0


"I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not going to lie, I'm not
---------

Prompt 1
Molly loves popcorn.
Generation 1


"I love popcorn," she said. "I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn. I love popcorn.
---------

Prompt 2
Tim rented a car to visit his ill mother.
Generation 2


"It was a very nice day," he said.

"It was a very nice day. It was a very nice day. It was a very nice day. It was a very nice day. It was a very nice day. It was a very nice day. It was a very nice day. I